In [ ]:
import torch
import torch.nn.functional as F
import timm
import torchvision.transforms as transforms
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Definir caminho do modelo treinado
MODEL_PATH = "model/xception_deepfake.pth"
# Diretórios com as imagens
TEST_DIR = "videos-teste/CELEB-DF/faces"

In [8]:
# Carregar o modelo treinado
model = timm.create_model("xception41.tf_in1k", pretrained=False, num_classes=2)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

XceptionAligned(
  (stem): Sequential(
    (0): ConvNormAct(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): ReLU(inplace=True)
      )
    )
    (1): ConvNormAct(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): ReLU(inplace=True)
      )
    )
  )
  (blocks): Sequential(
    (0): XceptionModule(
      (shortcut): ConvNormAct(
        (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (bn): BatchNormAct2d(
          128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
      )
      (stack): Sequential(
     

In [9]:
# 📌 Criar transformações para normalização
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Dimensão da entrada do Xception
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

NameError: name 'transforms' is not defined

In [ ]:
# 📌 Função para aplicar Grad-CAM em uma imagem
def apply_gradcam(image_path):
    # Carregar imagem e converter para RGB
    img = Image.open(image_path).convert("RGB")

    # Transformar imagem
    input_tensor = transform(img).unsqueeze(0).to(device)

    # 📌 Aplicar Grad-CAM++
    cam_extractor = SmoothGradCAMpp(model)  # Usando Smooth Grad-CAM++
    target_layer = list(model.named_children())[-2][1]  # Última camada convolucional do Xception
    cam_extractor._target_layer = target_layer

    # Obter saída do modelo e calcular ativação Grad-CAM
    output = model(input_tensor)
    class_idx = output.argmax().item()  # Pegar a classe predita
    activation_map = cam_extractor(class_idx, output)  # Gera o mapa de ativação

    # Converter imagem para array NumPy
    img_array = np.array(img)

    # Redimensionar o mapa de ativação para o tamanho da imagem original
    heatmap = cv2.resize(activation_map[0].cpu().numpy(), (img_array.shape[1], img_array.shape[0]))

    # Normalizar o heatmap para [0,1]
    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())

    # Aplicar heatmap na imagem original
    result = overlay_mask(Image.fromarray(img_array), Image.fromarray((heatmap * 255).astype(np.uint8)), alpha=0.5)

    # 📌 Exibir imagem original e heatmap
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].imshow(img)
    ax[0].set_title("Imagem Original")
    ax[0].axis("off")

    ax[1].imshow(result)
    ax[1].set_title(f"Grad-CAM - {class_idx}")
    ax[1].axis("off")

    plt.show()

In [1]:
full_name = "videos-teste/CELEB-DF/faces/Deepfake/id0_id26_0005/frame_105_face_1.jpg"

# 📌 Testar com uma imagem de face extraída
apply_gradcam(full_name)

NameError: name 'apply_gradcam' is not defined